# Исследование надежности заемщиков

**Описание проекта**

Наш заказчик — кредитный отдел банка. Заказчик строит модель кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку. Нам необходимо на основании статистики о платёжеспособности клиентов определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок, чтобы учесть эти выводы в модели.

**Цели и задачи проекта**

Цель проекта - опеределить как влияет семейное положение и количество детей заемщика на факт погашения кредита в срок.

Задачи проекта:
1. Провести загрузку и предобработку даннных  
2. Декомпозировать исходный датафрейм и категоризировать отдельные параметры  
3. Оценить влияние различных факторов на просрочку по кредиту

# Содержание
1. [Обзор данных](#start)<a id='start2'></a> 
2. [Заполнение пропусков](#step2)  
3. [Проверка данных на аномалии и исправления](#step3)     
4. [Изменение типов данных](#step4)  
5. [Удаление дубликатов](#step5)  
6. [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#step6)  
7. [Категоризация дохода](#step7)  
8. [Категоризация целей кредита](#step8)  
9. [Оценка зависимости просрочки от различных факторов](#step9)  
10. [Ответы на вопросы](#step10)  
11. [Общий вывод](#step11)  

## [Обзор данных](#start2)
<a id="start"></a>

In [ ]:
import pandas as pd

In [1]:
# выведем начало таблицы, чтобы посмотреть на данные и провести первичную оценку состава датафрейма
df.head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выводы по итогам оценки состава датафрейма:**
1. В колонке `days_employed` большинство значений записано с минусом. При этом встречаются положительные значения. Отрицательных значений больше, кажется, что положительные - это аномалии. Но количество дней не может быть отрицательным, поэтому, скорее всего, нам нужно менять отрицательные значения на положительные. 
2. Еще странно, что количество дней записано в дробном формате.
3. В колонке `eduacation` одни и те же значения записаны разным регистром. Это может привести к ошибкам при анализе. Необходимо это исправить.
4. Парные колонки `education` / `education_id` и `family_status` / `family_status_id` внутри себя несут одинаковый смысл. Целесообразно выделить их в отдельные датафреймы.
5. В колонке `purpose` имеются неявные дубликаты, например, покупка жилья / операции с жильем / покупка жилья для семьи и сыграть свадьбу / на проведение свадьбы. Необходимо это исправить.

In [2]:
# используем метод для просмотра структуры датафрейма
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Выводы по итогам оценки структуры датафрейма:**
1. Объем датафрейма существенный, руками не перебрать.
2. Названия колонок понятные и записаны корректно - в змеином регистре.
3. В колонках `days_employed` и `total_income` имеются пропуски. Причем их количество совпадает. Будем разбираться.


## [Заполнение пропусков](#start2)
<a id="step2"></a>

Проанализируем столбец `total_income` на предмет пропусков и их замены:
- посмотрим, в каких строках встречаются пропуски
- посчитаем количество строк с пропусками и их долю в общем количестве строк
- оценим, как пропуски влияют на качество данных
- решим, что делать с пропусками: удалить или заменить на какое-либо значение

In [3]:
# смотрим, в каких строках встречаются пропуски
df[df['total_income'].isna()].head(10) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


In [4]:
# считаем долю пропусков

# количество пропусков (строка для самопроверки)
display(len(df[df['total_income'].isna()])) 

# общее количество строк (строка для самопроверки)
display(len(df['total_income'])) 

# доля пропусков
len(df[df['total_income'].isna()])/len(df['total_income']) 

2174

21525

0.10099883855981417

**Вывод:**

Доля пропусков в колонке `total income` 10%. Это много. Удаление строк с пропусками в этой колонке может ухудшить результы работы с данными. Поэтому правильнее заменить пропуски.

Так как колонка содержит количественные значения, то менять пропуски надо либо на среднее значение, либо на медианное. По сравнению с медианой, среднее значение смещено в сторону самых часто встречающихся значений. Медиана исключает эту проблему и позволяет выбрать среднее всего диапазона доступных значений. Поэтому для замены пропусков выбираем медиану.

In [5]:
# меняем пропуски на медиану колонки
df['total_income'] = df['total_income'].fillna(value=df['total_income'].median()) 

# количество пропусков после замены (строка для самопроверки)
len(df[df['total_income'].isna()])

0

## [Проверка данных на аномалии и исправления](#start2)
<a id="step3"></a>

Проанализируем столбец `days_employed` на предмет аномалий.
- определим, в чем заключается аномалия
- попробуем объяснить причину аномалии
- устраним аномалии
- т.к. в колонке есть пропуски, заполним их медианой по столбцу, аналогично предыдущему этапу

В колонке `days_employed` встречаются отрицательные значения. Количество дней не может быть отрицаельным, поэтому можем считать это аномалией. Попробуем определить причины возникновения аномалии.

In [6]:
# посчитаем долю отрицательных значений в колонке `days_employed`

# количество отрицательных значений в колонке (строка для самопроверки)
display(len(df[df['days_employed'] <= 0]))

# общее количество значений в колонке (строка для самопроверки)
display(len(df['days_employed']))

len(df[df['days_employed'] <= 0]) / len(df['days_employed'])

15906

21525

0.7389547038327526

**Вывод:**

Почти 3/4 всех значений в колонке отрицательные. Если количество дней рассчитывается автоматически, как разница между датами, то, возможно, при заполнении даты путали местами.

In [7]:
# исправим аномалии - заменим отрицательные значения на положительные
df['days_employed'] = df['days_employed'].loc[df['days_employed'] <= 0] * (-1)

# количество отрицательных значений в колонке (строка для самопроверки)
display(len(df[df['days_employed'] <= 0]))

0

In [8]:
# заполним пропуски медианным значением
df['days_employed'] = df['days_employed'].fillna(value=df['days_employed'].median()) 

# количество пропусков после замены (строка для самопроверки)
len(df[df['days_employed'].isna()])

0

In [9]:
# еще раз проверим результат замены пропусков
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [10]:
# проверим наличие аномалий в других колонках
display(df['children'].value_counts())
display(df['dob_years'].value_counts())
display(df['gender'].value_counts())
display(df['income_type'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Осмотр данных показал, что аномалии есть почти во всех выбранных колонках:
- в колонке `children` есть отрицательные значения и значения 20. Скорее всего это ошибки. Мы не знаем природу этих ошибок, чтобы правильно выбрать значение для замены, хоть и похоже, что это опечатки. Т.к. их не много, мы проигнорируем их в дальнейшем анализе.
- в колонке `dob_years` есть нулевые значения. По сути это пропуски. Заменим их а среднее значение. Выбираем именно среднее, а не медиану, потому что средняя отражает наиболее активный возраст, когда люди могут и берут кредиты.
- в колонке `gender` есть нулевое значение. Т.к. оно одно и не влияет на общий результат, мы его пропустим.
- в колонке `income_type` кажется, что всё в порядке.

In [11]:
# заменим нулевые значения на среднюю в колонке dob_years
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].mean())

# переведем возраст в тип int. Хотел с делать одной строкой в строке выше, но вот так - df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].mean().astype(int)) - не сработало
df['dob_years'] = df['dob_years'].astype(int) 

# проверим, что получилось
df['dob_years'].value_counts()

35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
22    183
66    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

## [Изменение типов данных](#start2)
<a id="step4"></a>

Пока не совсем понимаю, что нам это даст, но в соответствии с инструкцией к проекту, заменим тип значений в колонке `total_income` c дробного на целочисленный.

In [12]:
df['total_income'] = df['total_income'].astype(int)

# проверим, что получилось
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int64  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## [Удаление дубликатов](#start2)
<a id="step5"></a>

Проведем работу с дубликатами:
- найдем и обработаем неявные дубликаты
- удалим явные дубликаты
- поясним, как выбирали метод для поиска и удаления дубликатов в данных
- приведем возможные причины появления дубликатов

In [13]:
# определим количество дубликатов методом duplicated
df.duplicated().sum()

54

In [14]:
# переведем все значения колонки `education` в нижний регистр
df['education'] = df['education'].str.lower()

# посмотрим, что получилось
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [15]:
# повторим то же самое с другими колонками типа строка
df['family_status'] = df['family_status'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()

# и посмотрим, что получилось
display(df['family_status'].unique())
display(df['income_type'].unique())
display(df['purpose'].unique())

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'не женат / не замужем'], dtype=object)

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [16]:
# проверим, как изменилось количество дубликатов после обработки неявных дубликатов
display(df.duplicated().sum())

# удалим дубликаты со сбросом индексов
df = df.drop_duplicates().reset_index(drop=True)

# проверим, что получилось
display(df.duplicated().sum())
df.info()

71

0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education         21454 non-null  object 
 4   education_id      21454 non-null  int64  
 5   family_status     21454 non-null  object 
 6   family_status_id  21454 non-null  int64  
 7   gender            21454 non-null  object 
 8   income_type       21454 non-null  object 
 9   debt              21454 non-null  int64  
 10  total_income      21454 non-null  int64  
 11  purpose           21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Выводы:**

На данном шаге, при поиске и удалении дубликатов мы использовали метод `duplicated`, потому что, в отличеи от метода `value_counts` его можно применить сразу ко всему датафрейму.

Одной из причин появления дубликатов могли стать повторные записи с целью исправить регистр в некоторых колонках.

## [Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма](#start2)
<a id="step6"></a>

Обработаем исходный датафрейм, сделаем его легче для обработки:
- выделим связанные столбцы в отдельные словари
- оставим в исходном датафрейме только идентификаторы значений в выделенных столбцах
- удалим из исходного датафрейма столбцы, не являющиеся идентификаторами

In [17]:
# проверим уникальные значения в колонках education и family_status
display(df['education'].value_counts())
display(df['family_status'].value_counts())

# создадим новую таблицу без колонок education и family_status. Можно перезаписать исходный датафрейм, но для подстраховки мы создадим новый.
df_stat = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]

# создадим списоки-словари для значений education и family_status
education_dict = df[['education_id', 'education']]
family_status_dict = df[['family_status_id', 'family_status']]

# удалим дубликаты в словарях
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)

# проверим, что получилось
display(education_dict)
display(family_status_dict)
df_stat.head(10)

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,1630.019381,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926.185831,27,0,1,M,компаньон,0,255763,покупка жилья
6,0,2879.202052,43,0,0,F,компаньон,0,240525,операции с жильем
7,0,152.779569,50,1,0,M,сотрудник,0,135823,образование
8,2,6929.865299,35,0,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188.756445,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи


## [Категоризация дохода](#start2)
<a id="step7"></a>

Категоризируем доход:
- определим группы дохода и создадим групповую кодировку (уже есть в инструкции к проекту)
- создадим функцию для присвоения кода каждой категории
- добавим в основной датафрейм колонку с кодом группы дохода

In [18]:
# создадим функицию для присвоения кода каждой категории
def total_income_group(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    else:
        return 'A'

# проверим, что получилось
print(total_income_group(20000), total_income_group(900000), total_income_group(10000000), sep='')
print('© WALL-E')

EBA
© WALL-E


In [19]:
# добавим в основной датафрейм колонку с кодом группы дохода
df_stat['total_income_category'] = df_stat['total_income'].apply(total_income_group)

# проверим, что получилось
df_stat.head()

/tmp/ipykernel_195/830421318.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stat['total_income_category'] = df_stat['total_income'].apply(total_income_group)


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,1630.019381,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


## [Категоризация целей кредита](#start2)
<a id="step8"></a>

Категоризируем цель кредита:

- создадим функцию для категоризации каждой цели
- добавим в основной датафрейм колонку с категорией

In [20]:
# посмотрим на имеющиеся значения целей кредита
display(df['purpose'].value_counts())

# создадим функцию для присвоения категоризации
def porpose_group(purpose):
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'жиль' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    if 'образ' in purpose:
        return 'получение образования'
    
# добавим в основной датафрейм колонку с новыми категориями цели кредита
df_stat['purpose_category'] = df_stat['purpose'].apply(porpose_group)

# проверим, что получилось
display(df_stat['purpose_category'].value_counts())
df_stat.info()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

операции с недвижимостью    10811
операции с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21454 non-null  int64  
 1   days_employed          21454 non-null  float64
 2   dob_years              21454 non-null  int64  
 3   education_id           21454 non-null  int64  
 4   family_status_id       21454 non-null  int64  
 5   gender                 21454 non-null  object 
 6   income_type            21454 non-null  object 
 7   debt                   21454 non-null  int64  
 8   total_income           21454 non-null  int64  
 9   purpose                21454 non-null  object 
 10  total_income_category  21454 non-null  object 
 11  purpose_category       21454 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


## [Оценка зависимости просрочки от различных факторов](#start2)
<a id="step9"></a>

In [21]:
# оценим зависимость между количеством детей и возвратом кредита в срок, посчитаем долю просрочек по долгу для каждой категории
df_stat_children = pd.pivot_table(
    df_stat, 
    index = 'children', 
    values = 'debt', 
    aggfunc = ['sum', 'count', 'mean']
).sort_values(by=('mean', 'debt'), ascending=False)
df_stat_children

,sum,count,mean
,debt,debt,debt
children,,,
20,8,76,0.105263
4,4,41,0.097561
2,194,2052,0.094542
1,444,4808,0.092346
3,27,330,0.081818
0,1063,14091,0.075438
-1,1,47,0.021277
5,0,9,0.000000


In [22]:
# оценим зависимость между семейным статусом и возвратом кредита в срок, посчитаем долю просрочек по долгу для каждой категории
df_stat_family_status = pd.pivot_table(df_stat, index = ['family_status_id'], values = ['debt'], aggfunc = ['sum', 'count'])
df_stat_family_status['conversion'] = df_stat_family_status['sum'] / df_stat_family_status['count']
df_stat_family_status.sort_values(by='conversion', ascending=False)

,sum,count,conversion
,debt,debt,
family_status_id,,,
4,274,2810,0.097509
1,388,4151,0.093471
0,931,12339,0.075452
3,85,1195,0.071130
2,63,959,0.065693


In [23]:
# оценим зависимость между уровнем дохода и возвратом кредита в срок, посчитаем долю просрочек по долгу для каждой категории
df_stat_total_income = pd.pivot_table(df_stat, index = ['total_income_category'], values = ['debt'], aggfunc = ['sum', 'count'])
df_stat_total_income['conversion'] = df_stat_total_income['sum'] / df_stat_total_income['count']
df_stat_total_income.sort_values(by='conversion', ascending=False)

,sum,count,conversion
,debt,debt,
total_income_category,,,
E,2,22,0.090909
C,1360,16016,0.084915
A,2,25,0.080000
B,356,5041,0.070621
D,21,350,0.060000


In [24]:
# оценим зависимость между целью кредита и возвратом кредита в срок, посчитаем долю просрочек по долгу для каждой категории
df_stat_purpose = pd.pivot_table(df_stat, index = ['purpose_category'], values = ['debt'], aggfunc = ['sum', 'count'])
df_stat_purpose['conversion'] = df_stat_purpose['sum'] / df_stat_purpose['count']
df_stat_purpose.sort_values(by='conversion', ascending=False)

,sum,count,conversion
,debt,debt,
purpose_category,,,
операции с автомобилем,403,4306,0.093590
получение образования,370,4013,0.092200
проведение свадьбы,186,2324,0.080034
операции с недвижимостью,782,10811,0.072334


## [Ответы на вопросы](#start2)
<a id="step10"></a>

**Вопрос 1:**

Есть ли зависимость между количеством детей и возвратом кредита в срок?

**Вывод 1:**

Зависимость риска просрочки от количества детей не линейная, но однозначно можно сказать, что наличие детей этот риск увеличивает.

**Вопрос 2:**

Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод 2:**

Самая высокая доля просрочек у людей состоящих в гражданском браке и одиноких (не женат/не замужем). Скорее всего, что два этих множества пересекаются. Таким образом, одинокие люди и люди, не регистрирующие свои отношения, несут высоки риск просрочки.

**Вопрос 3:**

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод 3:**

Самая высокая доля просрочек у заемщиков с минимальным доходом. Это выглядит логично, хотя таких замщиков очень мало. Скорее всего они не проходят по параметрам банка для выдачи кредитов.

Вторая по просрочкам - категория заемщиков со средним доходом. Это самая массовая категория. Для более эффективного риск-менеджемента было бы правильно разбить ее на подкатегории и посмотреть по каждой долю просрочки. Потому что заемщик с зарплатой 55 000 руб. может быть очень далеко по платежеспособности, социальному статусу и присущим рискам от заемщика с зарплатой 190 000 руб.

**Вопрос 4:**

Как разные цели кредита влияют на его возврат в срок?

**Вывод 4:**

Самая высокая доля просрочек по автокредитам и на образовательные цели. Вероятно, заемщик по автокредиту чаще может поддаться эмоциям при покупке авто и не рассчитать свои финансовые возможности. При займе на образовательные цели есть риск, что заемщик не успеет конвертировать свое образование в трудоустройство и зарплату и соответственно вернуть долг.

## [Общий вывод](#start2)
<a id="step11"></a>

Заказчик поставил задачу разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. По итогам исследования можно сделать вывод, что да, оба этих фактора имеют влияние.

Портрет наиболее надежного заемщика - это человек, состоящий в браке и не имеющий детей.
Наименее надежный заемщик - это человек, никогда ранее не регистрировавший брак и имеющий несколько детей.

Кроме того, в процессе анализа мы выяснили, что целесообразно рассмотреть и другие факторы для включения в скоринг-модель. Например, доход заемщика и цель кредита. 

Доход замщика является важнейшим фактором оценки его платежеспособности, но необходимо очень внимательно подходить к его категоризации и оценке. Скорее всего, понадобится отдельный более глубокий анализ. 

Напротив, цель кредита имеет относительно прозрачную взаимосвязь с риском возврата.